<font face="XB Zar" size=5><div dir=rtl align=center>
<font face="IranNastaliq" size=5>
به نام خدا
</font>
<br>
<font size=3>
دانشگاه صنعتی شریف - دانشکده مهندسی کامپیوتر
</font>
<br>
<font color=blue size=5>
تمرین امتیازی بخش بهینه سازی
</font><br>
<font color=blue size=5>
نوت بوک زبان جولیا
</font>
# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" />
<br>
محمد معین صمدی آزاد 
<br>
400105093



<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="red" size=5>**نصب جولیا و پکیج های لازم**</font>


In [127]:
# @title
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


## reload the page !

In [128]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [129]:
] add COBREXA

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [130]:
] add JuMP

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [131]:
] add GLPK

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="red" size=5>**مقدمه**</font>

<font face="XB Zar" size=4><div dir=rtl>
در این تمرین قصد داریم با یکی از کاربردهای CSP آشنا شویم. فرض کنید شما میخواهید دارویی را تولید کنید که سنتز آن از صفر یا بسیار پرهزینه است یا به حدی پیچیده است که در عمل ممکن نیست اما یک نوع باکتری در طبیعت وجود دارد که توانایی سنتز دارو را به طور طبیعی دارد. شما میتوانید با استخراج واکنش های انجام شده در باکتری و مهندسی آن ها موجب شوید باکتری داروی مطلوب شما را  به میزان زیادی تولید کند. برای مثال انسولین که به طور روزانه توسط بیماران مبتلا به دیابت مصرف میشود با استفاده از باکتری های مهندسی شده تولید میشود.
 <font face="XB Zar" size=4><div dir=rtl>
 در مراحل مختلفی از این روند مانند پیش پردازش داده های واکنش ها، میتوان از CSP بهره برد که در این تمرین قصد داریم با آن آشنا شویم.

<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="red" size=5>**معرفی المان های مساله**</font>

<font face="XB Zar" size=4><div dir=rtl>
همانطور که گفته شده ما یک باکتری داریم که واکنشهای درون باکتری در آزمایشگاه استخراج شده و داده های آن در اختیار ما قرار گرفته است. این داده ها شامل موارد زیر هستند:
<br>
<br>
n: تعداد واکنش ها
<br>
m: مجموع تعداد ماده های متمایز در تمام واکنش ها
<br>
S: یک ماتریس m*n که سطر i,j از آن بیان میدارد در واکنش j ام چند واحد از ماده ی i مصرف یا تولید میشود. مقدار منفی بیانگر مصرف و مقدار مثبت بیانگر تولید است. (درواقع این ماتریس ضرایب استکیومتری واکنش هاست، فراورده ها مثبت و واکنش دهنده ها منفی)
<br>
ub: یک ماتریس به طول n که upper bound یا کران بالای rate واکنش ها را مشخص میکند
<br>
lb: یک ماتریس به طول n که lower bound یا کران پایین rate واکنش ها را مشخص میکند
<br>
Reactions: ماتریس به طول n، نام واکنش ها
<br>
Metabolites: ماتریس به طول m، نام ماده ها (متابولیت ها)


In [132]:
using COBREXA
using JuMP
using GLPK

In [133]:
struct models
           S
           Metabolites
           Reactions
           Genes
           m
           n
           lb
           ub
       end

<font face="XB Zar" size=4><div dir=rtl>
<font color="blue">
**سوال:**
<br>
<font color="black">
چگونه میتوان از روی مقادیر lb و ub تشخیص داد واکنش ها برگشت پذیر هستند یا نه؟

<font color="pink">
پاسخ
<br>
در صورتی که مینیمم منفی باشد، یک واکنش برگشت پذیر است

<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="red" size=5>**مثال**</font>

<font face="XB Zar" size=4><div dir=rtl>
ساده ترین حالت CSP قابل حل با این داده ها این هست که با توجه
به محدودیت های دنیای واقعی، یک حالت از rate های ممکن برای واکنش را به دست آوریم:
ماتریس V را تعریف میکنیم:
<br>
  یک ماتریس به طول n است که درایه ی iام نظیر rate واکنش iام است.
  <div dir=ltr>
<br>
Find $V$
<br>
Subject to :
<br>
$Lb[j]<v[j]<ub[j]$
<br>
$SV=0$


<font face="XB Zar" size=4><div dir=rtl>
<font color="blue">
**سوال:**
<br>
<font color="black">
شرط SV=0 بیانگر چیست؟

<font color="pink">
پاسخ
<br>
در این حالت تغیر همه مواد صفر و در نتیجه در وضعیت استیبل هستیم.

<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="red" size=5>**پیش پردازش داده ها**</font>
<br>
<font color="red">
یافتن و حذف واکنش های بلاک شده

<font face="XB Zar" size=4><div dir=rtl>
همانطور که میدانیم داده های دریافت شده از آزمایشگاه همیشه داده های تمیزی نیست و نیازبه پیش پردازش دارد. یک نمونه از مشکلات موجود در این داده ها وجود واکنش های "بلاک شده" هستند. ممکن است آزمایشگاه واکنش هایی را گزارش کرده باشد که تحت هیچ شرایطی امکان انجام آن وجود نداشته باشد و در هر حالت ممکنی ازمقداردهی rate ها، rate این واکنش همیشه صفر باشد. یافتن و حذف این واکنش ها موجب میشود فضای حالت ما کوچکتر شود و حل مسائل بعدی ساده تر شود.

<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="red">
تعریف دقیق واکنش های بلاک شده:
<br>
<font color="black">
واکنشی که در هیچ حالتی از فضای حالت تمام rateهای ممکن نتواند rate غیرصفر داشته باشد، در هیچ شرایطی اجرا نمیشود و بلاک شده است. توجه کنید برای واکنش های دوطرفه این شرط باید از هر دوسمت (رفت و برگشت) برقرار باشد تا واکنش بلاک شده تلقی شود.


<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="red" size=5>**حل مساله ی یافتن واکنش های بلاک شده:**</font>
<br>
<font color="black">
در ادامه با استفاده از CSP قصد داریم واکنش های بلاک شده را پیدا کنیم. توضیحی در مورد هرکدام از توابع مطرح شده و ورودی و خروجی آنها در ادامه آمده که باید این توابع را تکمیل کنید. میتوانید ورودی و خروجی توابع را با خواست خود تغییر دهید.


<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="blue">
تابع loadmyModel
<br>
<font color="black">
این تابع مدل واکنش را دانلود میکند و در فرمت یک استراکت برمیگرداند، نیازی به تغییر دادن این تابع نیست.

In [134]:
function loadmyModel(name)
  !isfile(name) &&
    download(string("http://bigg.ucsd.edu/static/models/",name), name);
  model = load_model(StandardModel, name)
  m=n_metabolites(model)
  n=n_reactions(model)
  myModel=models(stoichiometry(model),metabolites(model),reactions(model),genes(model),m,n,bounds(model)[1],bounds(model)[2])
  return myModel
end

loadmyModel (generic function with 1 method)

<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="blue">
تابع if_rev
<br>
<font color="black">
این تابع نام (یا یک ویژگی) از واکنش را دریافت میکند و درخروجی مشخص که واکنش برگشت پذیر هست یا نه

In [135]:
function if_rev(reaction, model)
  if model.lb[reaction] < 0
    return 1
  end
  return 0
end

if_rev (generic function with 1 method)

<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="blue">
تابع homogenize
<br>
<font color="black">
این تابع مدل را دریافت کرده و آن را همگن میکند یعنی ub را برای تمام واکنش ها +M و lb را برای واکنش های برگشت پذیر -M و برای واکنش های برگشت ناپذیر 0 قرار میدهد.

In [136]:
function homogenize(model,M)
  for i in 1:model.n
    model.ub[i] = M
    if if_rev(i, model)== 1
       model.lb[i] = -M
    else
        model.lb[i] = 0
    end
  end
  return model
end


homogenize (generic function with 1 method)

<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="blue">
تابع irreversable_solve
<br>
<font color="black">
این تابع برای هر واکنش برگشت ناپذیر با حل یک CSP، مشخص میکند آن واکنش بلاک شده هست یا نه و در خروجی واکنش های برگشت ناپذیر بلاک شده را برمیگرداند

In [137]:
function irreversable_solve(myModel)
x = Vector{Int64}()
for i in 1:myModel.n
  r = myModel.Reactions[i]
  if if_rev(i, myModel) == 0
    m = Model(GLPK.Optimizer)
    @variable(m,v[1:myModel.n])
    @objective(m,Max,v[i])
    for j = 1:myModel.n
        @constraint(m, myModel.lb[j]<= v[j] <=myModel.ub[j])
    end
    @constraint(m, constraint1, myModel.S * v .== zeros(myModel.m, 1))
    # Solving the optimization problem
    JuMP.optimize!(m)
    if JuMP.value(v[i]) == 0
      push!(x, i)
    end
   end
end
for xx in x
  println(myModel.Reactions[xx])
end
return x
end

irreversable_solve (generic function with 1 method)

<font face="XB Zar" size=4><div dir=rtl id="1">
<font color="blue">
تابع reversable_solve
<br>
<font color="black">
این تابع برای هر واکنش برگشت پذیر با حل دو CSP، مشخص میکند آن واکنش بلاک شده هست یا نه و در خروجی واکنش های برگشت پذیر بلاک شده را برمیگرداند

In [143]:
function reversable_solve(myModel)
x = Vector{Int64}()
for i in 1:myModel.n
  r = myModel.Reactions[i]
  if if_rev(i, myModel) == 1
    m = Model(GLPK.Optimizer)

    @variable(m,v[1:myModel.n])
    @objective(m,Max,v[i])
    for j = 1:myModel.n
        @constraint(m, myModel.lb[j]<= v[j] <=myModel.ub[j])
    end
    @constraint(m, constraint1, myModel.S * v .== zeros(myModel.m, 1))
    # Solving the optimization problem
    JuMP.optimize!(m)
    xnxx =JuMP.value(v[i])

    m_s = Model(GLPK.Optimizer)

    @variable(m_s,v_s[1:myModel.n])
    @objective(m_s,Min,v_s[i])
    for j = 1:myModel.n
        @constraint(m_s, myModel.lb[j]<= v_s[j] <=myModel.ub[j])
    end
    @constraint(m_s, constraint1, myModel.S * v_s .== zeros(myModel.m, 1))
    # Solving the optimization problem
    JuMP.optimize!(m_s)
    if JuMP.value(v_s[i]) == 0 && xnxx == 0
      push!(x, i)
    end

   end
end
for xx in x
  println(myModel.Reactions[xx])
end
return x
end

reversable_solve (generic function with 1 method)

In [139]:
name="e_coli_core.xml"
print(">",name,"\n")
myModel=loadmyModel(name)
M=1000000

>e_coli_core.xml


1000000

In [140]:
#if_rev("a reaction name")

In [144]:
myModel=homogenize(myModel,M)
print("#irreversable blocked reactios:\n")
irreversable_solve(myModel)
print("#reversable blocked reactios:\n")
reversable_solve(myModel)

#irreversable blocked reactios:
R_EX_fru_e
R_EX_fum_e
R_EX_gln__L_e
R_EX_mal__L_e
R_FRUpts2
R_FUMt2_2
R_GLNabc
R_MALt2_2
#reversable blocked reactios:


Int64[]

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.